## Comparison Between Different Number of Layers
---

### Hidden Layer Settings in Experiment

In [ ]:
dim3 = (1024, 512, 64)
dim5 = (2048, 1024, 512, 256, 64)
dim7 = (2048, 1024, 512, 256, 128, 64, 32)
dim9 = (4096, 2048, 1024, 512, 256, 128, 64, 32, 16)
hiddens = [dim3, dim5, dim7, dim9]

### 1. Load modules and datasets

In [ ]:
# Set path of this notebook to the root directory
import os
os.chdir(os.path.join(os.getcwd(), '..'))
print 'Current working directory', os.getcwd()

In [ ]:
# For auto-reloading external modules
%load_ext autoreload
%autoreload 2

# Import modules
import pandas as pd
import numpy as np
import data_utils as du
import neural_network as nn
from sklearn.model_selection import KFold


# Prepare datasets for processing
du.maybe_pickle('train.csv')
du.maybe_pickle('test.csv')

train_dataset = du.load_dataset('train')
test_dataset = du.load_dataset('test')

print
print 'Train dataset has shape =', train_dataset[0].shape
print 'Test dataset has shape =', test_dataset[0].shape

### 2. Configure Network

In [ ]:
optimizer = 'RMSPropOptimizer'
learning_rate = 0.001
num_steps = 5000
batch_size = 256
val_folds = 7    # Only fold 0 will be used (no cross-validation)

# Configure the following lines in affine_layer_variables() in layer_utils.py
# w_init = tf.divide(tf.truncated_normal([matrix_dim0, matrix_dim1], stddev=0.1), tf.sqrt(matrix_dim0 / 2.0))
# b_init = tf.zeros([matrix_dim1])

### 3. Prepare Testing Variables and Functions

In [ ]:
def run_network(hidden_dim):
    
    kf = KFold(n_splits=val_folds)
    train_idx, val_idx = kf.split(train_dataset[0]).next()
    
    
    clf = nn.NeuralNetwork(hidden_dim=hidden_dim, optimizer=optimizer, 
                 learning_rate=learning_rate, num_steps=num_steps,
                 batch_size=batch_size)
    
    train = du.get_batch(train_dataset, indices=train_idx)
    val = du.get_batch(train_dataset, indices=val_idx)
    
    clf.fit(train, val, 'log/hidden/dim' + str(len(hidden_dim)) + '/')
    train_acc = clf.accuracy(train)
    val_acc = clf.accuracy(val)

    print 'accuracy on train dataset = %.4f' % train_acc
    print 'accuracy on validation dataset = %.4f' % val_acc

### 4. Run Network for Every Hidden Layers Configuration

In [ ]:


for hidden in hiddens:
    print "Running %d layers" % len(hidden)
    run_network(hidden)
    print 

### 5. Plotting Results

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def getData(path):
    """
        Returns data from CSV file exported from Tensorboard. The returned data is an np array.
    """
    data = pd.read_csv(path).iloc[:, 1:].as_matrix()
    return data

In [ ]:
hiddens = [dim3, dim5, dim7]
colors = ['#0049BA', '#06BA00', '#F9F630']
for idx, dim in enumerate(hiddens):
    loss_data = getData('plotting/data_csv/hidden/loss/dim' + str(len(dim)) + '.csv')
    x, y = loss_data[:, 0], loss_data[:, 1]
    plt.plot(x, y, color=colors[idx], label='%d-layer' % len(dim), zorder=-idx)
plt.title('Loss Function for Different Number of Hidden Layers')
plt.legend()
plt.xlabel('steps')
plt.ylabel('loss')
plt.axis([0, 5000, 0, 1.5])
plt.grid(True)
plt.show()

In [ ]:
# Plot val accuracies for depth 3, 5 and 7
hiddens = [dim3, dim5, dim7]
colors = ['#0049BA', '#06BA00', '#F9F630']
for idx, dim in enumerate(hiddens):
    val_data = getData('plotting/data_csv/hidden/val_acc/dim' + str(len(dim)) + '.csv')
    x, y = val_data[:, 0], val_data[:, 1]
    plt.plot(x, y, color=colors[idx], label='%d-layer' % len(dim), zorder=-idx)
plt.title('Validation Accuracy for Different Number of Hidden Layers')
plt.legend(bbox_to_anchor=(1, 0.3))
plt.xlabel('steps')
plt.ylabel('validation accuracy')
plt.axis([0, 5000, 0.3, 1])
plt.grid(True)
plt.show()